<a href="https://colab.research.google.com/github/Anna321321321321/Binary-Classification-for-Imbalanced-Data-Set/blob/master/Copy_of_Copy_of_Regression_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regression
Reference: part of the following material (particularly on kernels) was adapted from [Prof. Mark Schmidt's](https://www.cs.ubc.ca/~schmidtm/) CPSC 540 notes. 

#### Importing Packages

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from sklearn import linear_model
from sklearn import preprocessing

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


!pip install PyDrive

%matplotlib inline

Red wine file ID '1b_3YfJs1tI9oa_dmTy5QZcRXH3kGGcrp' \\
White wine file ID '1yjqY04haDk-BQfZdCWmz0ATXk7WdoSkD'

### Background
Consider a set of $i = 1, \dots, n$ samples where each sample contains a set of $j = 1, \dots, d$ features, $x_{ij}$, and a label $y_i$. With linear regression our label is a linear function of our features, i.e., 
$$ \hat{y}_i=w_1x_{i1}+w_2x_{i2}+\cdots+w_dx_{id} = \sum_{j=1}^d w_jx_{ij} = w^Tx_i $$
where $w_j$ are the weights or regression coefficients of $x_i$.

### Least squares objective
A common way to determine the regression coefficients is by minimizing the sum of squared errors between the predicted label ($\hat{y}_i = w^Tx_i$) and the true label ($y_i$), i.e., 
$$f(w) = \frac{1}{2}\sum_{i=1}^n(w^Tx_i-y_i)^2 = \frac{1}{2}\begin{Vmatrix} Xw-y \end{Vmatrix}^2 $$
with $X \in \mathbb{R}^{n \times d}$, $y \in \mathbb{R}^{n \times 1}$, $w \in \mathbb{R}^{d \times 1}$ and $f(w)$ is commonly referred to as the loss function. Calculating the optimal $w$ can be performed by setting the gradient to zero (i.e., $\nabla f(w)=0$) which results in $$w = (X^TX)^{-1}(X^Ty).$$

### Adding a y-intercept bias
Given our previous linear function of features we are restricted to having predictions that pass through the origin, i.e., $\hat{y}_i = 0$ when $x_i = 0$. This is resolved by adding a y-intercept bias, i.e., $\hat{y}_i = \beta + w^Tx_i$. The y-intercept bias can simply be considered an additional regression coefficient that is multiplied by one as opposed to $x_{ij}$. Incorporating this y-intercept into our original expression is as simple as adding a column of ones at the beginning of $X$ and appending $\beta$ in $w$ such that $\beta$ is the first regression coefficient, i.e., $w = [\beta,w_1,\dots,w_d]^T$.

#### Example: Predicting red wine quality
We will analyze a dataset containing properties of variants of the Portuguese "Vinho Verde" red wine. Each sample in the dataset contains a score for quality (between 0 and 10) based on sensory data and a set of measured properties including the fixed acidity, volatile acidity, citric acid content, residual sugar content, chloride content, free sulfur dioxide, total sulfur dioxide, density, pH and sulphates. Our objective is to generate a model that can provide accurate predictions of the quality score based on the measured properties. Additional information regarding this dataset can be found [here](http://archive.ics.uci.edu/ml/datasets/Wine+Quality).

Source: [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/index.php)

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':"1yjqY04haDk-BQfZdCWmz0ATXk7WdoSkD"})   # replace the id with id of file you want to access
downloaded.GetContentFile('WhiteWineQuality.csv')   

In [0]:
#Load the data
data = pd.read_csv('WhiteWineQuality.csv',sep=';')

In [0]:
print(data.head())

   fixed acidity  volatile acidity  citric acid  ...  sulphates  alcohol  quality
0            7.0              0.27         0.36  ...       0.45      8.8        6
1            6.3              0.30         0.34  ...       0.49      9.5        6
2            8.1              0.28         0.40  ...       0.44     10.1        6
3            7.2              0.23         0.32  ...       0.40      9.9        6
4            7.2              0.23         0.32  ...       0.40      9.9        6

[5 rows x 12 columns]


In [0]:
#Select labels and features
Y = data['quality']
X = data.values
X = X[:,:-1]

# Collect training/testing data
N = X.shape[0]
print('Total samples:',N)
perm = np.random.permutation(np.arange(N))
Xp = X[perm]
Yp = Y[perm]
n = int(0.8*N)
print('Training samples:',n)
print('Testing samples:',N-n)
X_train = Xp[:n,:]
Y_train = Yp[:n]
X_test = Xp[n:,:]
Y_test = Yp[n:]

Total samples: 4898
Training samples: 3918
Testing samples: 980


In [0]:
# Perform simple linear regression with intercept
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
regr = linear_model.LinearRegression()
# Train the model
regr.fit(X_train,Y_train)
# Make predictions on testing datas
Y_pred = regr.predict(X_test)
# Analyze results
print('Y-intercept:', regr.intercept_)
print('Regression coefficients:', regr.coef_)
print("Mean squared error: %.3f" % mean_squared_error(Y_test,Y_pred))
print("Mean absolute error: %.3f" % mean_absolute_error(Y_test,Y_pred))

Y-intercept: 139.65781287271957
Regression coefficients: [ 5.81343998e-02 -1.87027211e+00  2.93693334e-02  7.69194793e-02
 -1.69971051e-01  3.33482768e-03 -2.74494591e-04 -1.39558009e+02
  6.43694137e-01  6.26673196e-01  2.03109233e-01]
Mean squared error: 0.550
Mean absolute error: 0.569


### Changing the basis
If we suspect that our label is a non-linear function of our features we can change our basis to include polynomials of a higher degree. For example if we have two features and want to use a quadratic basis we can have $$\hat{y}_i = \beta + w_1x_{i1} + w_2x^2_{i1} + w_3x_{i2} + w_4x^2_{i2}.$$
Our original regression coefficients had $d=2$, after adding the y-intercept it became $d=3$ and finally after including the quadratic bases we have $d=5$. The new feature matrix, denoted $\Phi(X)$, is simply an expanded version of $X$ with added columns for the quadratic terms.

Using the general polynomial basis (i.e., $[1,x,x^2,\dots,x^p]$) results in lower degree polynomials being nested within higher degree polynomials. As the degree of the general polynomial basis increases the model becomes more sensitive to the training data and therefore provides a lower training error (i.e., bias). Unfortunately high sensitivity to the training data can cause over-fitting wherein our model lacks the generality to perform well on the testing data. Using different training sets can yield a large variety of models, i.e., our model has a high variance with respect to the training data. For more information on the tradeoff between complex and general models the interested reader is referred to the [bias variance tradeoff](http://www.cs.cornell.edu/courses/cs578/2005fa/CS578.bagging.boosting.lecture.pdf). 

In [0]:
# Using polynomial bases
from sklearn.preprocessing import PolynomialFeatures
# Repeat with different degrees
poly = PolynomialFeatures(degree=5)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.fit_transform(X_test)
regr_poly = linear_model.LinearRegression()
regr_poly.fit(X_train_poly,Y_train)
Y_pred_poly = regr_poly.predict(X_test_poly)
# Analyze results
# DO NOT PRINT REGRESSION COEFFICIENTS IF degree>2
print('Regression coefficients:', regr_poly.coef_)
print("Mean squared error: %.3f" % mean_squared_error(Y_test,Y_pred_poly))
print("Mean absolute error: %.3f" % mean_absolute_error(Y_test,Y_pred_poly))

Regression coefficients: [-4.38994825 14.15757255  4.2120547  ... -1.04969244 -0.35684826
  0.01496601]
Mean squared error: 289.536
Mean absolute error: 3.420


### L2-Regularization
One common method for controlling model complexity is through L2-regularization which penalizes the squared L2-norm of the regression coefficients, i.e., $\begin{Vmatrix}w \end{Vmatrix}^2$. Our minimization problem becomes 
$$ \underset{w \in \mathbb{R}^d}{\text{arg min}} \quad \frac{1}{2}\begin{Vmatrix}Xw-y \end{Vmatrix}^2 + \frac{\lambda}{2}\begin{Vmatrix}w \end{Vmatrix}^2$$
where the regularization parameter $\lambda$ is a tuning parameter that controls the weight of regularization and ultimately the resulting model complexity. Including the regularization term in our loss function, $f(w)$, and setting the gradient to zero yields the following expression for the regression coefficients:
$$w = (X^TX+\lambda I_d)^{-1}(X^Ty).$$
If $n << d$ the solution for $w$ can be computed faster using an equivalent expression for $w$, i.e., 
$$ w = X^T(XX^T+\lambda I_n)^{-1}y,$$
because $(X^TX) \in \mathbb{R}^{d \times d}$ whereas $(XX^T) \in \mathbb{R}^{n \times n}$. In addition to likely improving the test error, regularization is recommended because it allows for a non-invertible $X^TX$.

### Standardization
By penalizing the L2-norm of the regression coefficients it becomes necessary to standardize the scales of the features. For example, if the values of $x_{i1}$ are on the order of $10^5$ and the values of $x_{i2}$ are on the order of $10^{-1}$, regularization will ensure that the regression coefficient $w_1$ corresponding to the large feature is forced to be small even if this degrades our model accuracy. To address this issue we standardize the scale of each feature by subtracting the mean and dividing by the standard deviation of said feature, i.e.,
$$x_{ij} = \frac{x_{ij}-\mu_j}{\sigma_j},$$
where 
$$ \mu_j = \frac{1}{n}\sum_{i=1}^n x_{ij} \quad \text{and} \quad \sigma_j = \sqrt{\frac{1}{n}\sum_{i=1}^n(x_{ij}-\mu_j)^2}.$$
This standardization ensures that regularization of $w$ has a similar effect on each feature. Similarly, the scale of the label $y_i$ is often standardized such that it is on the same scale as the standardized features, i.e.,
$$y_i = \frac{y_i-\mu_y}{\sigma_y}.$$

In [0]:
# Standardize features
X_train_std = preprocessing.scale(X_train)
X_test_std = preprocessing.scale(X_test)
#Y_train_std = preprocessing.scale(Y_train)
#Y_test_std = preprocessing.scale(Y_test)
#Create polynomial bases
X_train_poly_std = poly.fit_transform(X_train_std)
X_test_poly_std = poly.fit_transform(X_test_std)
# Regularization with standardized data
regr_ridge = linear_model.Ridge(alpha=0.1)
regr_poly_ridge = linear_model.Ridge(alpha=0.1)
# alpha serves the role of lamdba, tinker with it to improve results
regr_ridge.fit(X_train_std,Y_train)
regr_poly_ridge.fit(X_train_poly_std,Y_train)
Y_pred_ridge = regr_ridge.predict(X_test_std)
Y_pred_poly_ridge = regr_poly_ridge.predict(X_test_poly_std)
print("MSE with regularization: %.3f" % mean_squared_error(Y_test,Y_pred_ridge))
print("MAE with regularization: %.3f" % mean_absolute_error(Y_test,Y_pred_ridge))
print("MSE with polynomial bases and regularization: %.3f" % mean_squared_error(Y_test,Y_pred_poly_ridge))
print("MAE with polynomial bases and regularization: %.3f" % mean_absolute_error(Y_test,Y_pred_poly_ridge))

MSE with regularization: 0.411
MAE with regularization: 0.497
MSE with polynomial bases and regularization: 0.404
MAE with polynomial bases and regularization: 0.496


### Additional Analysis 

In [0]:
# Lets analyze our results in terms of explained variance (1 is best score, lower values are worse)
from sklearn.metrics import explained_variance_score
print("Explained variance score (ridge regression): %.3f" % explained_variance_score(Y_test,Y_pred_ridge))

Explained variance score (ridge regression): 0.374


In [0]:
# Analyze our ridge regression coefficients
print('Ridge regression coefficients:', regr_ridge.coef_)
print(min(abs(regr_ridge.coef_)))
print(max(abs(regr_ridge.coef_)))

Ridge regression coefficients: [ 0.0435567  -0.20750831 -0.03849282  0.02744457 -0.08671946  0.05086615
 -0.12115427 -0.02650881 -0.05709872  0.15611237  0.28047024]
0.026508808053880853
0.2804702367534059


In [0]:
# Largest absolute coefficient corresponds to alcohol content
# Smallest absolute coefficient corresponds to citric acid content

In [0]:
print("Explained variance score (all features): %.3f" % explained_variance_score(Y_test,Y_pred_ridge))
# Remove the alcohol content feature from our data and fit a new model to the remaining features
X_train_std_NA = np.delete(X_train_std,10,1)
X_test_std_NA = np.delete(X_test_std,10,1)
regr_ridge_NA = linear_model.Ridge(alpha=30)
regr_ridge_NA.fit(X_train_std_NA,Y_train)
Y_pred_ridge_NA = regr_ridge_NA.predict(X_test_std_NA)
print("Explained variance score (no alcohol): %.3f" % explained_variance_score(Y_test,Y_pred_ridge_NA))

# Remove the citric acid content feature from our data and fit a new model to the remaining features
X_train_std_NC = np.delete(X_train_std,2,1)
X_test_std_NC = np.delete(X_test_std,2,1)
regr_ridge_NC = linear_model.Ridge(alpha=30)
regr_ridge_NC.fit(X_train_std_NC,Y_train)
Y_pred_ridge_NC = regr_ridge_NC.predict(X_test_std_NC)
print("Explained variance score (no citric acid): %.3f" % explained_variance_score(Y_test,Y_pred_ridge_NC))

Explained variance score (all features): 0.374
Explained variance score (no alcohol): 0.315
Explained variance score (no citric acid): 0.374


# Exercises and additional data sets

#### Exercise #1

From a choice of $\alpha = [0.001, 0.1, 1, 10, 100, 1000]$ determine the best choice of $\alpha$ for minimizing the test error of predictions using ridge regression with both linear and polynomial bases.

## Predicting the quality of white wine

In WhiteWineQuality.csv you can find a set of data for predicting the quality of Portuguese white wine based on the same features as described before. Please note, the white wine file ID is '1yjqY04haDk-BQfZdCWmz0ATXk7WdoSkD' and it can be loaded as we loaded the red wine data set above.

 #### Exercise #2 
Load the white wine dataset, divide the data into separate training/testing sets and standardize the scale of the features.

#### Exercise #3
Using the processed white wine data, perform regression with a polynomial basis and determine the degree of the polynomial basis (from 1-5) that minimizes the test error.

#### Exercise #4 
Model the white wine data using ridge regression (linear basis) but use only 3 features. Carefully select these three features such that they maximize the explained variance of the resulting predictions. Compare the explained variance score of your 3-feature model to the explained variance score of the same type of model but with all of the features.

## Bonus: Advanced Topics


### Kernel Trick
Assume we have trained our L2-regularized model with the alternative expression for $w$ introduced before. With this model we want to predict new labels $\hat{y}$ using test data $\hat{X}$, i.e., 
$$\hat{y} = \hat{X}w = \hat{X}X^T(XX^T+\lambda I_n)^{-1}y.$$
Defining $K=XX^T$ and $\hat{K} = \hat{X}X^T$ yields
$$\hat{y} = \hat{K}(K+\lambda I_n)^{-1}y,$$
where $K$ is commonly referred to as the Gram matrix and it contains the inner products between all of the training examples, i.e., $k(x_{i:},x_{j:}) = x^T_{i:}x_{j:}$. From the previous expression for $\hat{y}$ we can see that computing these inner products allows us to no longer rely on $x_{i:}$ and $x_{j:}$.

Recall the polynomial basis from before but now consider including feature interactions (i.e., $x_{i1}x_{i2}$, $x_{i1}x^2_{i2}$, etc.). Adding a column for each new feature results in a large feature matrix $\Phi(X)$ with $O(d^p)$ terms where $p$ is the degree of the polynomial basis and $d$ is the number of features. The kernel trick provides an efficient method for handling problems with high degree bases and many features. For example, consider a problem with two samples, $x_{i:}$ and $x_{j:},$ and two features, $x_{i:}=(x_{i1},x_{i2})$ and $x_{j:}=(x_{j1},x_{j2})$, where we have the following second degree basis
$$ \phi(x_{i:})=\begin{pmatrix}x^2_{i1},\sqrt{2}x_{i1}x_{i2},x^2_{i2} \end{pmatrix}.$$
To get the Gram matrix we need to compute the inner product $\phi(x_{i:})^T\phi(x_{j:})$ which can be accomplished without explicitly forming $\phi(x_{i:})$ and $\phi(x_{j:})$, i.e.,
$$\begin{align} 
\phi(x_{i:})^T\phi(x_{j:}) &= \begin{bmatrix}x^2_{i1} & \sqrt{2}x_{i1}x_{i2} & x^2_{i2}  \end{bmatrix}\phi(x_{j:}) \\ 
&= x^2_{i1}x^2_{j1}+2x_{i1}x_{i2}x_{j1}x_{j2}+x^2_{i2}x^2_{j2} \\
&=\begin{pmatrix}x_{i1}x_{j1}+x_{i2}x_{j2}\end{pmatrix}^2 = \begin{pmatrix}\sum_{k=1}^d x_{ik}x_{jk}\end{pmatrix}^2 = \begin{pmatrix}x^T_{i:}x_{j:}\end{pmatrix}^2.
\end{align}$$
If we wish to include all degree-$p$ monomials with a bias we simply raise our expression to the $p$-th power and add a constant inside the power which yields the following expressions for $K$ and $\hat{K}$:
$$k(x_{i:},x_{j:}) = \begin{pmatrix}1+x^T_{i:}x_{j:}\end{pmatrix}^p \quad \text{and} \quad \hat{k}(\hat{x}_{i:},x_{j:}) = \begin{pmatrix}1+\hat{x}^T_{i:}x_{j:}\end{pmatrix}^p.$$
These expressions can then be used to make prediction as $\hat{y} = \hat{K}(K+\lambda I_n)^{-1}y$ with a cost of $O(n^2d+n^3)$ as opposed to $O(d^p)$. Another advantage of the kernel trick is that $k(x_i,x_j)$ can be interpreted as a measure of similarity between objects which allows us to apply regression even when we don't necessarily know the features. 

### Radial Basis Functions
The polynomial basis from before is an example of a parametric model wherein the size of the model is independent of the number of training examples $n$. Other parametric models include bases such as exponentials, logarithms and various trigonometric functions. Alternatively, non-parametric models grow with the number of training examples. Radial basis functions (RBFs) are a particular type of non-parametric bases that depend on distances to the training points. One of the more common kernels is the Gaussian-RBF kernel, i.e.,
$$k(x_{i:},x_{j:}) = \text{exp}\begin{pmatrix}-\frac{\begin{Vmatrix}x_{i:}-x_{j:}\end{Vmatrix}^2}{\sigma^2} \end{pmatrix},$$
where the variance $\sigma^2$ is a user-defined parameter that controls the relative contribution of training points $x_{j:}$ with regards to their distance from $x_{i:}$.

In [0]:
from sklearn.kernel_ridge import KernelRidge

RBF_KernelRidge = KernelRidge(alpha=0.1,kernel='rbf',gamma=0.022)
RBF_KernelRidge.fit(X_train_std,Y_train)
Y_pred_RBF = RBF_KernelRidge.predict(X_test_std)
print("MSE with RBF kernel: %.3f" % mean_squared_error(Y_test,Y_pred_RBF))
print("MAE with RBF kernel: %.3f" % mean_absolute_error(Y_test,Y_pred_RBF))
print("Explained variance score with RBF kernel: %.3f" % explained_variance_score(Y_test,Y_pred_RBF))

MSE with RBF kernel: 0.405
MAE with RBF kernel: 0.489
Explained variance score with RBF kernel: 0.460


#### Exercise #5

From a choice of $\alpha = [0.001, 0.1, 1, 10, 100, 1000]$ and $\gamma = [0.001, 0.1, 1, 10, 100, 1000]$ determine the best choices of $\alpha$ and $\gamma$ for minimizing the MSE/MAE of predictions using kernel ridge regression (KRR) as before. Hint: Use GridSearchCV
